In [ ]:
#===============================================================
# Þessi hluti býr til functions sem eru notuð seinna í kóðanum.
#===============================================================

def ydot(yi, L=2, g=9.81):
    return [yi[1], (-g/L)*m.sin(yi[0])]

def eulerstep(yi, h, func, L=2, g=9.81):
    return [x + h*func(yi,L,g)[i] for i,x in enumerate(yi)]

def euler(y0, T, n, L=2, g=9.81):
    h = T/n
    y = [y0]

    for i in range(n-1):
        y.append(eulerstep(y[i], h, ydot, L, g))

    theta = []
    omega = []

    for x in y:
        theta.append(x[0])
        omega.append(x[1])

    return theta, omega

def f(y):
    theta, omega = y
    g, L = 9.81, 2
    d_theta = omega
    d_omega = -(g/L)*m.sin(theta)
    return np.array([d_theta,d_omega])

def f2(start, m1=1, m2=1, l1=2, l2=2, g=9.81):
    
    # start = [y1, y2, y3, y4] = [θ1, θ2, ω1, ω2]
    y1 = start[0]
    y2 = start[1]
    y3 = start[2]
    y4 = start[3]

    delta = y2-y1 # Δ = θ2 - θ1

    a = m2*l1*(y3**2)*m.sin(delta)*m.cos(delta)
    b = m2*g*m.sin(y2)*m.cos(delta)
    c = m2*l2*(y4**2)*m.sin(delta)
    d = (m1+m2)*g*m.sin(y1)
    e = (m1+m2)*l1
    f = m2*l1*((1-m.cos(2*delta))/2)
    func1 = (a+b+c-d)/(e-f)
    i = m2*l2*(y4**2)*m.sin(delta)*m.cos(delta)
    j = (m1+m2)*(g*m.sin(y1)*m.cos(delta)-l1*(y3**2)*m.sin(delta)-g*m.sin(y2))
    k = (m1+m2)*l2
    l = m2*l2*((1-m.cos(2*delta))/2)
    func2 = (-i+j)/(k-l)

    return np.array([y3, y4, func1, func2])

def RKsolver_Y4(y0, T, n, f, y_final=''): # modified for vector y 4x1

    h = T / n  # time step

    y = y0

    # y0 = [y1, y2, y3, y4] = [θ1, θ2, ω1, ω2]
    y1 = [y0[0]]
    y2 = [y0[1]]
    y3 = [y0[2]]
    y4 = [y0[3]]

    for _ in range(n):
        k1 = f(y)
        k2 = f(y + 0.5*h*k1)
        k3 = f(y + 0.5*h*k2)
        k4 = f(y + h*k3)

        y = y + (h/6) * (k1 + 2*k2 + 2*k3 + k4)
        y1.append(y[0])
        y2.append(y[1])
        y3.append(y[2])
        y4.append(y[3])
    if y_final == 'y':
        return np.array(y1[-1], y2[-1], y3[-1], y4[-1])
    else:
        return np.array([y1, y2, y3, y4]) # θ1:list, θ2:list, ω1:list, ω2:list

def save_animation(anim, filename="vid.mp4", fps=30):
    
    choice = input("Make video file? (Y/N): ").strip().lower()
    if choice == "y":
        Writer = animation.writers['ffmpeg']
        writer = Writer(fps=fps, metadata=dict(artist="Me"), bitrate=1800)
        anim.save(filename, writer=writer)
        print("Saved to:", os.path.abspath(filename))
    else:
        print("Video not saved.")

def mjaaa(start, m1=1, m2=1, l1=2, l2=2, g=9.81):
    
    # start = [y1, y2, y3, y4] = [θ1, θ2, ω1, ω2]
    y1 = start[0]
    y2 = start[1]
    y3 = start[2]
    y4 = start[3]

    delta = y2-y1 # Δ = θ2 - θ1

    a = m2*l1*(y3**2)*m.sin(delta)*m.cos(delta)
    b = m2*g*m.sin(y2)*m.cos(delta)
    c = m2*l2*(y4**2)*m.sin(delta)
    d = (m1+m2)*g*m.sin(y1)
    e = (m1+m2)*l1
    f = m2*l1*((1-m.cos(2*delta))/2)
    func1 = (a+b+c-d)/(e-f)
    i = m2*l2*(y4**2)*m.sin(delta)*m.cos(delta)
    j = (m1+m2)*(g*m.sin(y1)*m.cos(delta)-l1*(y3**2)*m.sin(delta)-g*m.sin(y2))
    k = (m1+m2)*l2
    l = m2*l2*((1-m.cos(2*delta))/2)
    func2 = (-i+j)/(k-l)

    return np.array([y3, y4, func1, func2])
    return np.array([y3, y4, func1, func2]) 

def make_multi_pendulums_plt(pendulums_list, theta, filename='vid.mp4', fps=30):
    """
    pendulums_list: List of pendulums. 
                    Each element is a list/tuple of two tuples: [(x1, y1), (x2, y2)]
                    representing mass 1 and mass 2 coordinates arrays.
    theta: Used only for frames count len(theta).
    """

    plt.close("all")
    fig = plt.figure()
    ax = fig.add_subplot(
        111, autoscale_on=False,
        xlim=(-5, 5), ylim=(-5, 5),
        aspect="equal"
    )
    plt.xlabel("Lengdareining")
    plt.ylabel("Lengdareining")
    fig.tight_layout()

    FPS_PLAY = fps
    
    # Colors to distinguish different double pendulums
    colors = ['b', 'r', 'g', 'c', 'm', 'y', 'k', 'orange', 'purple', 'brown']

    line_objects = []
    circle_objects = []

    # Initialize all pendulums
    for idx, p_data in enumerate(pendulums_list):
        # p_data should be [(x1, y1), (x2, y2)]
        (x1, y1), (x2, y2) = p_data
        
        col = colors[idx % len(colors)]

        # Mass 1 (connected to origin)
        line1, = ax.plot([0, x1[0]], [0, y1[0]], color=col, lw=2, alpha=0.7)
        bob1 = ax.add_patch(plt.Circle((x1[0], y1[0]), 0.1, fc=col, zorder=3))

        # Mass 2 (connected to Mass 1)
        line2, = ax.plot([x1[0], x2[0]], [y1[0], y2[0]], color=col, lw=2, alpha=0.7)
        bob2 = ax.add_patch(plt.Circle((x2[0], y2[0]), 0.1, fc=col, zorder=3))

        # Store in flat lists for the animation update
        # Structure: [p1_l1, p1_l2, p2_l1, p2_l2, ...]
        line_objects.extend([line1, line2])
        circle_objects.extend([bob1, bob2])

    def init():
        for line in line_objects:
            line.set_data([], [])
        for circ in circle_objects:
            circ.center = (-1000, -1000)
        return line_objects + circle_objects

    def animate(i):
        # Update every pendulum
        for idx, p_data in enumerate(pendulums_list):
            (x1, y1), (x2, y2) = p_data
            
            # Retrieve the specific objects for this pendulum
            l1_obj = line_objects[idx*2]
            l2_obj = line_objects[idx*2 + 1]
            b1_obj = circle_objects[idx*2]
            b2_obj = circle_objects[idx*2 + 1]

            # Current coordinates
            if i < len(x1):
                cx1, cy1 = x1[i], y1[i]
                cx2, cy2 = x2[i], y2[i]

                l1_obj.set_data([0, cx1], [0, cy1])
                b1_obj.center = (cx1, cy1)

                l2_obj.set_data([cx1, cx2], [cy1, cy2])
                b2_obj.center = (cx2, cy2)
        
        return line_objects + circle_objects

    anim = animation.FuncAnimation(
        fig, animate, frames=len(theta),
        interval=1000/FPS_PLAY, blit=True,
        init_func=init
    )

    plt.show()

    choice = input("Make video file? (Y/N): ").strip().lower()
    if choice == "y":
        Writer = animation.writers["ffmpeg"]
        writer = Writer(fps=fps, metadata=dict(artist="Me"), bitrate=1800)
        anim.save(filename, writer=writer)
        print("Saved to:", filename)
    else:
        print("Video not saved.")

def makeDoublePendulumCoords(y0, T, n, l1=2, l2=2):
    '''
    y0=[theta1_0, theta2_0, omega1_0, omega2_0]
    '''

    theta1, theta2, omega1, omega2 = RKsolver_Y4(y0, T, n, mjaaa)
    theta_both = [theta1,theta2]

    x1 = [l1*m.sin(theta) for theta in theta1]
    y1 = [-l1*m.cos(theta) for theta in theta1]
    x2 = [l2*m.sin(theta_both[0][i])+l2*m.sin(theta_both[1][i]) for i in range(len(theta1))]
    y2 = [-l1*m.cos(theta_both[0][i])-l2*m.cos(theta_both[1][i]) for i in range(len(theta1))]
    return ((x1, y1), (x2, y2))
    #f.make_plt([(x1, y1),(x2, y2)], theta1, filename, fps)

def getDistance(x1, y1, x2, y2):
    return m.sqrt((x2-x1)**2 + (y2-y1)**2)

def RKsolver_Y6(y0, T, n, f, y_final=''): # modified for vector y 6x1

    h = T / n  # time step

    y = y0

    # y0 = [y1, y2, y3, y4, y5, y6] = [θ1, θ2, θ3, ω1, ω2, ω3]
    y1 = [y0[0]]
    y2 = [y0[1]]
    y3 = [y0[2]]
    y4 = [y0[3]]
    y5 = [y0[4]]
    y6 = [y0[5]]

    for _ in range(n):
        k1 = f(y)
        k2 = f(y + 0.5*h*k1)
        k3 = f(y + 0.5*h*k2)
        k4 = f(y + h*k3)

        y = y + (h/6) * (k1 + 2*k2 + 2*k3 + k4)
        y1.append(y[0])
        y2.append(y[1])
        y3.append(y[2])
        y4.append(y[3])
        y5.append(y[4])
        y6.append(y[5])

    if y_final == 'y':
        return np.array([y1[-1], y2[-1], y3[-1], y4[-1], y5[-1], y6[-1]])
    else:
        return np.array([y1, y2, y3, y4, y5, y6]) # θ1:list, θ2:list, θ3:list, ω1:list, ω2:list, ω3:list

def omegaDot(theta1, theta2, omega1, omega2, m1=1, m2=1, L1=2, L2=2, g=9.81):
    delta = theta2 - theta1
    #Teljari í omega1dot, aðskilið með + eða -
    T11 = m2*L1*(omega1**2)*m.sin(delta)*m.cos(delta)
    T12 = m2*g*m.sin(theta2)*m.cos(delta)
    T13 = m2*L2*(omega2**2)*m.sin(delta)
    T14 = -(m1+m2)*g*m.sin(theta1)

    #Nefnari í omega1dot, aðskilið með + eða -
    N1 = (m1+m2)*L1
    N2 = -m2*L1*(m.cos(delta)**2)
    
    omega1dot = (T11+T12+T13+T14)/(N1+N2)

    #Teljari í omega2dot, aðskilið með + eða -
    T21 = -m2*L2*(omega2**2)*m.sin(delta)*m.cos(delta)

def f3(state, m1=1.0, m2=1.0, m3=1.0, l1=2.0, l2=2.0, l3=2.0, g=9.81):

    th1, th2, th3, w1, w2, w3 = state
    th = np.array([th1, th2, th3], dtype=float)
    w = np.array([w1, w2, w3], dtype=float)
    ls = [l1, l2, l3]
    ms = [m1, m2, m3]

    # Jacobian
    Js = []
    for r in range(3):
        J = np.zeros((2,3), dtype=float)
        for j in range(r+1):
            J[0, j] = ls[j] * m.cos(th[j])
            J[1, j] = ls[j] * m.sin(th[j])
        Js.append(J)

    M = np.zeros((3,3), dtype=float)
    for r in range(3):
        Jr = Js[r]
        M += ms[r] * (Jr.T @ Jr)

    dM = [np.zeros((3,3), dtype=float) for _ in range(3)]
    for k in range(3):
        for r in range(3):
            if k <= r:
                dJr = np.zeros((2,3), dtype=float)
                dJr[0, k] = - ls[k] * m.sin(th[k]) 
                dJr[1, k] =   ls[k] * m.cos(th[k]) 
                Jr = Js[r]
                dM_k_r = ms[r] * (dJr.T @ Jr + Jr.T @ dJr)
                dM[k] += dM_k_r

    dMdt = np.zeros((3,3), dtype=float)
    for k in range(3):
        dMdt += dM[k] * w[k]

    q_vec = np.zeros(3, dtype=float)
    for k in range(3):
        q_vec[k] = float(w @ (dM[k] @ w))

    dV = np.zeros(3, dtype=float)
    for k in range(3):
        sum_m = 0.0
        for r in range(k, 3):
            sum_m += ms[r]
        dV[k] = sum_m * g * ls[k] * m.sin(th[k])

    RHS = - (dMdt @ w - 0.5 * q_vec + dV)

    try:
        acc = np.linalg.solve(M, RHS)
    except np.linalg.LinAlgError:
        acc = np.linalg.lstsq(M, RHS, rcond=None)[0]

    return np.array([w1, w2, w3, acc[0], acc[1], acc[2]], dtype=float)

def RKsolverLotkaVolterra(y0, T, n, f):

    y = np.array(y0, dtype=float)
    h = T / n  # time step
    theta = [y0[0]]
    omega = [y0[1]]

    for _ in range(n):
        k1 = f(y)
        k2 = f(y + 0.5*h*k1)
        k3 = f(y + 0.5*h*k2)
        k4 = f(y + h*k3)

        y = y + (h/6) * (k1 + 2*k2 + 2*k3 + k4)
        theta.append(y[0])
        omega.append(y[1])

    return theta, omega

def make_plt(*args, filename='vid.mp4', fps=30, frame=5):
    """
    Unified make_plt that supports:

    - File 1 style:
        make_plt(x, y, theta)

    - Files 2/3/4 style:
        make_plt(pendulums, theta, filename='vid.mp4', fps=30, frame=5)

      where pendulums is:
        [(x1, y1)]                     # single pendulum
        [(x1, y1), (x2, y2)]           # double pendulum
        [(x1, y1), (x2, y2), (x3, y3)] # triple pendulum
    """

    # --- Decode arguments to a common form ---

    if len(args) == 3:
        # File 1 style: make_plt(x, y, theta)
        x, y, theta = args
        pendulums = [(x, y)]  # wrap as a single-segment system

    elif len(args) == 2:
        # Files 2/3/4 style: make_plt(pendulums, theta, ...)
        pendulums, theta = args
    else:
        raise TypeError(
            "make_plt expected either (x, y, theta) or (pendulums, theta, ...)"
        )

    num_p = len(pendulums)  # number of segments (1, 2, or 3)

    # --- Figure and axes setup ---

    plt.close("all")
    fig = plt.figure()
    ax = fig.add_subplot(
        111, autoscale_on=False,
        xlim=(-frame, frame), ylim=(-frame, frame),
        aspect="equal"
    )
    plt.xlabel("Lengdareining")
    plt.ylabel("Lengdareining")
    fig.tight_layout()

    FPS_PLAY = fps

    line_objects = []
    circle_objects = []

    # --- Initialize segments (similar logic to files 2–4) ---

    for idx, (x_seg, y_seg) in enumerate(pendulums):
        if idx == 0:
            # First mass connected to origin
            x0, y0 = 0.0, 0.0
        else:
            # Subsequent masses connected to the previous mass
            x_prev, y_prev = pendulums[idx - 1]
            x0, y0 = x_prev[0], y_prev[0]

        line, = ax.plot([x0, x_seg[0]], [y0, y_seg[0]], "b-", lw=2)
        bob = ax.add_patch(
            plt.Circle((x_seg[0], y_seg[0]), 0.1, fc="r", zorder=3)
        )

        line_objects.append(line)
        circle_objects.append(bob)

    # --- Animation init function (same idea as before) ---

    def init():
        for line in line_objects:
            line.set_data([], [])
        for circ in circle_objects:
            circ.center = (-1000, -1000)
        return line_objects + circle_objects

    # --- Animation update function ---

    def animate(i):
        x_joint, y_joint = 0.0, 0.0  # start at origin

        for idx, (x_seg, y_seg) in enumerate(pendulums):
            x_i = x_seg[i]
            y_i = y_seg[i]

            line = line_objects[idx]
            bob = circle_objects[idx]

            line.set_data([x_joint, x_i], [y_joint, y_i])
            bob.center = (x_i, y_i)

            # Next segment attaches to this mass
            x_joint, y_joint = x_i, y_i

        return line_objects + circle_objects

    # --- Create animation ---

    anim = animation.FuncAnimation(
        fig, animate, frames=len(theta),
        interval=1000 / FPS_PLAY, blit=True,
        init_func=init
    )

    plt.show()

    choice = input("Make video file? (Y/N): ").strip().lower()
    if choice == "y":
        Writer = animation.writers["ffmpeg"]
        writer = Writer(fps=fps, metadata=dict(artist="Me"), bitrate=1800)
        anim.save(filename, writer=writer)
        print("Saved to:", filename)
    else:
        print("Video not saved.")

    return anim

x = 2
print(x)